In [18]:
import imp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
from tqdm import tqdm

In [34]:
data_folder = "final_experiment/"
stem = "test_graphs/"
plot_folders = [stem+"compas/", stem+"compas_synth/", stem+"simple_synth/"]
#sns.set(rc={'figure.figsize':(12,10)})
sns.set_style(style="darkgrid")
font = {'family': 'normal',
        'weight': 'bold',
        'size': 25}

plt.rc('font', **font)
#missing_pct = [0, 10,25,50,75, 90]
missing_pct = [0,
               50]
NAME_KEYS = {"coldel": "Column deletion",
             "cca": "CCA",
             "mice_def": "Chained Equation",
             "mean": "Mean",
             "log_reg": "Logistic Regression",
             "svm": "SVM",
             "knn": "KNN",
             "rf_cat": "Random Forest",
             "eosum": "Sum of Equalised Odds",
             "spd": "Statistical Parity Difference",
             "acc": "Accuracy",
             "fair_reg_99": "Fair imputation lambda = 0.99",
             "fair_reg_95": "Fair imputation lambda = 0.95",
             "fair_reg_75": "Fair imputation lambda = 0.75",
             "fair_reg_50": "Fair imputation lambda = 0.50"}

In [35]:
def load(name):
    with open(Path(data_folder+name), 'r') as f:
        return json.load(f)


In [36]:
#compas_priors = load("compas_priors.json")
compas = load("compas.json")

In [37]:
compas['robust_Truepriors_count|is_Caucasian']["Averaged results"]

{'mcar|spd|log_reg|fair_reg_50': [0.25300778862747064, 0.25403438895510055],
 'mcar|eosum|log_reg|fair_reg_50': [0.4778618177964015, 0.47931584045895276],
 'mcar|acc|log_reg|fair_reg_50': [0.6282768777614143, 0.6282474226804129],
 'mcar|tpr0|log_reg|fair_reg_50': [0.5984828657235501, 0.5916471189085314],
 'mcar|tpr1|log_reg|fair_reg_50': [0.4711413585296437, 0.4639821293180914],
 'mcar|tnr0|log_reg|fair_reg_50': [0.6632997257504896, 0.6708208958662515],
 'mcar|tnr1|log_reg|fair_reg_50': [0.5308806635271945, 0.5380604925598629],
 'mcar|tprd|log_reg|fair_reg_50': [0.12734150719390616, 0.12766498959043987],
 'mcar|tnrd|log_reg|fair_reg_50': [0.13241906222329514, 0.13276040330638872],
 'mcar|spd|log_reg|log_reg': [0.25300778862747064, 0.25183921239906104],
 'mcar|eosum|log_reg|log_reg': [0.4778618177964015, 0.4751098091643167],
 'mcar|acc|log_reg|log_reg': [0.6282768777614143, 0.6284536082474231],
 'mcar|tpr0|log_reg|log_reg': [0.5984828657235501, 0.5984828657235501],
 'mcar|tpr1|log_reg|l

In [5]:
len(compas) - len(compas_priors)

3

In [38]:
IMPUTATIONS = [ "fair_reg_50", "log_reg", "cca", "mean", "mice_def", "coldel"]

In [39]:
sns.load_dataset("tips")

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [40]:
[1,2,3]*3

[1, 2, 3, 1, 2, 3, 1, 2, 3]

In [41]:
def plotting(df, params, folder_path):
    savepath = folder_path+"/"
    if not os.path.isdir(Path(savepath)):
        os.mkdir(Path(savepath))    
    for miss in tqdm(['mar', 'mcar']):
        for model in ["log_reg", "knn", "rf_cat"]:
            for metric in ["eosum", "acc", "spd"]:
                full_data = {"Missing percentage": missing_pct*len(IMPUTATIONS),
                            NAME_KEYS[metric]: [],
                            "Imputation method": []}
                columns = ["Missing percentage", NAME_KEYS[metric], "Imputation method"]
                for imp in IMPUTATIONS:
                    key = "|".join([miss,metric,model,imp])
                    data = df[params]["Averaged results"][key]
                    full_data["Imputation method"]+=[NAME_KEYS[imp]]*len(data)
                    full_data[NAME_KEYS[metric]]+=data
                    #full_data["Missing percentage"].append([NAME_KEYS[imp]]*len(data))
                #print(full_data)
                plotting_data = pd.DataFrame(full_data, columns = columns)
                #print(plotting_data.head())
                fig = plt.gcf()
                fig.set_size_inches(30, 17.5)
                sns.lineplot(x="Missing percentage", y=NAME_KEYS[metric], hue = "Imputation method", data = plotting_data)
                #fig.legend(bbox_to_anchor=(1.2,1))
                plt.title("Performance of "+NAME_KEYS[model]+" on the COMPAS dataset")
                plt.savefig(Path(savepath+miss+"_"+model+"_"+metric+".png"))
                plt.clf()



In [42]:
for key in compas.keys():
    plotting(compas, key, plot_folders[0]+"-".join(key.split("|")))

  0%|          | 0/2 [00:00<?, ?it/s]findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Sum of Equalised Odds': [0.4778618177964015, 0.15394491235550933, 0.4778618177964015, 0.32121619708947036, 0.4778618177964015, 0.18552414857262597, 0.4778618177964015, 0.6381619058326267, 0.4778618177964015, 0.35669466911246267, 0.8706375028126828, 0.8706375028126828], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Regression', 'Logistic Regression', 'CCA', 'CCA', 'Mean', 'Mean', 'Chained Equation', 'Chained Equation', 'Column deletion', 'Column deletion']}
{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Accuracy': [0.6282768777614143, 0.609641629847816, 0.6282768777614143, 0.6266470299459995, 0.6282768777614143, 0.6038389788905258, 0.6282768777614143, 0.623318605792833, 0.6282768777614143, 0.6215611192930786, 0.5967697594501724, 0.5967697594501724], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Re

 50%|█████     | 1/2 [00:04<00:04,  4.07s/it]

{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Sum of Equalised Odds': [0.4778618177964015, 0.47931584045895276, 0.4778618177964015, 0.4751098091643167, 0.4778618177964015, 0.4716673759520478, 0.4778618177964015, 0.4538872321256258, 0.4778618177964015, 0.4673728522118957, 0.8706375028126828, 0.8706375028126828], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Regression', 'Logistic Regression', 'CCA', 'CCA', 'Mean', 'Mean', 'Chained Equation', 'Chained Equation', 'Column deletion', 'Column deletion']}
{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Accuracy': [0.6282768777614143, 0.6282474226804129, 0.6282768777614143, 0.6284536082474231, 0.6282768777614143, 0.6283259695630834, 0.6282768777614143, 0.6278546882670599, 0.6282768777614143, 0.6279528718703982, 0.5967697594501724, 0.5967697594501724], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Reg

  0%|          | 0/2 [00:00<?, ?it/s]

...

{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Sum of Equalised Odds': [0.4778618177964015, 0.15456215031805862, 0.4778618177964015, 0.2542439760899343, 0.4778618177964015, 0.2863193850763625, 0.4778618177964015, 0.8178256498008966, 0.4778618177964015, 0.18943314405971473, 0.8706375028126828, 0.8706375028126828], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Regression', 'Logistic Regression', 'CCA', 'CCA', 'Mean', 'Mean', 'Chained Equation', 'Chained Equation', 'Column deletion', 'Column deletion']}
{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Accuracy': [0.6282768777614143, 0.6121256750122734, 0.6282768777614143, 0.6230338733431522, 0.6282768777614143, 0.4991654393716249, 0.6282768777614143, 0.5934511536573398, 0.6282768777614143, 0.6071575846833585, 0.5967697594501724, 0.5967697594501724], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Re

 50%|█████     | 1/2 [00:06<00:06,  6.17s/it]

{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Sum of Equalised Odds': [0.4778618177964015, 0.47931584045895276, 0.4778618177964015, 0.4751098091643167, 0.4778618177964015, 0.4716673759520478, 0.4778618177964015, 0.4538872321256258, 0.4778618177964015, 0.4673728522118957, 0.8706375028126828, 0.8706375028126828], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Regression', 'Logistic Regression', 'CCA', 'CCA', 'Mean', 'Mean', 'Chained Equation', 'Chained Equation', 'Column deletion', 'Column deletion']}
{'Missing percentage': [0, 50, 0, 50, 0, 50, 0, 50, 0, 50, 0, 50], 'Accuracy': [0.6282768777614143, 0.6282474226804129, 0.6282768777614143, 0.6284536082474231, 0.6282768777614143, 0.6283259695630834, 0.6282768777614143, 0.6278546882670599, 0.6282768777614143, 0.6279528718703982, 0.5967697594501724, 0.5967697594501724], 'Imputation method': ['Fair imputation lambda = 0.50', 'Fair imputation lambda = 0.50', 'Logistic Reg

100%|██████████| 2/2 [00:10<00:00,  5.05s/it]


<Figure size 2160x1260 with 0 Axes>

In [16]:
def plot_acc_vs_fair(df, params, folder_path):
    savepath = folder_path+"/"
    if not os.path.isdir(Path(savepath)):
        os.mkdir(Path(savepath))    
    for miss in tqdm(['mar', 'mcar']):
        for model in ["log_reg", "knn", "rf_cat"]:
            full_data = {NAME_KEYS["acc"]: [],
                         NAME_KEYS["eosum"]:[],
                        "Imputation method": []}
            columns = ["Accuracy", "Fairness", "Imputation method"]
            for imp in IMPUTATIONS:
                key_acc = "|".join([miss,"acc",model,imp])
                key_eosum = "|".join([miss,"eosum",model,imp])
                data_acc = df[params]["Averaged results"][key_acc]
                data_eosum = df[params]["Averaged results"][key_eosum]
                full_data["Imputation method"]+=[NAME_KEYS[imp]]*len(data_acc)
                full_data[NAME_KEYS["acc"]]+=data_acc
                full_data[NAME_KEYS["acc"]]+=data_eosum
                #full_data["Missing percentage"].append([NAME_KEYS[imp]]*len(data))
                #print(full_data)
                print(full_data)
                plotting_data = pd.DataFrame(full_data, columns = columns)
                #print(plotting_data.head())
                fig = plt.gcf()
                fig.set_size_inches(30, 17.5)
                sns.scatterplot(x=NAME_KEYS["acc"], y=NAME_KEYS["eosum"], hue = "Imputation method", data = plotting_data)
                #fig.legend(bbox_to_anchor=(1.2,1))
                plt.title("Performance of "+NAME_KEYS[model]+" on the COMPAS dataset")
                plt.savefig(Path(savepath+miss+"_"+model+"_acc_vs_fair"+".png"))
                plt.clf()

In [17]:
for key in compas_priors.keys():
    plot_acc_vs_fair(compas, key, plot_folders[0]+"-".join(key.split("|")))

  0%|          | 0/2 [00:00<?, ?it/s]

{'Accuracy': [0.6287432498772705, 0.6266568483063328, 0.6191703485517919, 0.6081246931762396, 0.6111929307805597, 0.6099656357388317, 0.47085727605964245, 0.2502948145024114, 0.07347996546071546, 0.10055948261686537, 0.12785381041358956, 0.14131982439258303], 'Sum of Equalised Odds': [], 'Imputation method': ['Fair imputation lambda = 0.95', 'Fair imputation lambda = 0.95', 'Fair imputation lambda = 0.95', 'Fair imputation lambda = 0.95', 'Fair imputation lambda = 0.95', 'Fair imputation lambda = 0.95']}


ValueError: All arrays must be of the same length